In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -q -y psi4 python=3.7 -c psi4

# install rdkit
!conda install -q -y rdkit python=3.7 -c rdkit

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loader" error.
!pip install distributed

--2022-04-24 11:48:35--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-24 11:48:35--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M   155MB/s    in 0.5s    

2022-04-24 11:48:35 (155 MB/s) - ‘Miniconda3-latest-Linux

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [3]:
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole
# check rdkit version
print('rdkit version: ', rdBase.rdkitVersion)

rdkit version:  2020.09.1


In [4]:
## convert SMILES to xyz format
def smi2xyz_neutral(smiles):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles)) # make mol object and add Hydrogen
    AllChem.EmbedMolecule(mol, AllChem.ETKDGv3()) # expand to 3D
    AllChem.MMFFOptimizeMolecule(mol) # structure optimization by MMFF
    #xyz = Chem.MolToXYZBlock(mol)
    conf = mol.GetConformer(-1)

    plus = smiles.count('+')
    minus = smiles.count('-')
    charge = plus - minus + 0 # total charge
    
    multi = (charge%2) + 1 # 2S+1 = 2*((1/2)*(charge%2)) + 1
    
    xyz = str(charge) + " " + str(multi)
    for atom, (x,y,z) in zip(mol.GetAtoms(), conf.GetPositions()):
        xyz += '\n'
        xyz += '{} {} {} {}'.format(atom.GetSymbol(), x, y, z)
        #xyz += '{}\t{}\t{}\t{}'.format(atom.GetSymbol(), x, y, z)
        
    return xyz

In [54]:
smiles = 'N'

In [55]:
xyz_neutral = smi2xyz_neutral(smiles)
mol_neutral = psi4.geometry(xyz_neutral)
print(mol_neutral.save_string_xyz())

0 1
 N    0.001106865266   -0.000355364543    0.069977118449
 H   -0.479869650073   -0.809299412367   -0.320704452239
 H   -0.469942122764    0.818247928455   -0.312596347974
 H    0.934432599128   -0.004010956655   -0.338985794105



In [56]:
## convert SMILES to xyz format
def smi2xyz_oxidized(smiles):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles)) # make mol object and add Hydrogen
    AllChem.EmbedMolecule(mol, AllChem.ETKDGv3()) # expand to 3D
    AllChem.MMFFOptimizeMolecule(mol) # structure optimization by MMFF
    #xyz = Chem.MolToXYZBlock(mol)
    conf = mol.GetConformer(-1)

    plus = smiles.count('+')
    minus = smiles.count('-')
    charge = plus - minus + 1 # total charge
    
    multi = (charge%2) + 1 # 2S+1 = 2*((1/2)*(charge%2)) + 1
    
    xyz = str(charge) + " " + str(multi)
    for atom, (x,y,z) in zip(mol.GetAtoms(), conf.GetPositions()):
        xyz += '\n'
        xyz += '{} {} {} {}'.format(atom.GetSymbol(), x, y, z)
        #xyz += '{}\t{}\t{}\t{}'.format(atom.GetSymbol(), x, y, z)
        
    return xyz

In [57]:
xyz_oxidized = smi2xyz_oxidized(smiles)
mol_oxidized = psi4.geometry(xyz_oxidized)
print(mol_oxidized.save_string_xyz())

1 2
 N    0.000097851814   -0.000242273571    0.069986298561
 H   -0.390497790323    0.856035832313   -0.320633067716
 H   -0.545813272962   -0.764125251574   -0.326024520605
 H    0.934951475906   -0.088544346885   -0.325756557696



In [58]:
psi4.core.set_output_file("log.txt")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

# input (PCM Solver)
pcm = '''
units = angstrom
medium
{
    solvertype = iefpcm
    solvent = acetonitrile
}
cavity
{
    type = gepol
    scaling = True
    radiiset = uff
    mode = implicit
}
'''
# acetonitrile solvet (epsilon = 37.5)

#psi4.set_options({'pcm': True})

#psi4.pcm_helper(pcm)
#RHF, ROHF, UHF, CUHF, RKS, UKS (Default: RHF)
#psi4.set_options({'pcm': True, 'reference': 'UHF'}) # HF, MP2 
#level = 'HF/STO-3G' # RHF, ROHF, UHF, CUHF
#level = 'MP2/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'MP2.5/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'MP3/6-31G' # RHF, ROHF, UHF, CUHF
#level = 'CCSD(T)/cc-pVTZ' # RHF, ROHF, UHF, CUHF

psi4.set_options({'pcm': True, 'reference': 'UKS'}) # DFT
level = 'B3LYP/6-31+G(d,p)' # RKS, UKS

scf_neutral_e, wfn = psi4.energy(level, return_wfn = True, molecule=mol_neutral)
print('energy at the {} level of theory:\t{:.4f} [Ha]'.format(level, scf_neutral_e))

energy at the B3LYP/6-31+G(d,p) level of theory:	-56.5717 [Ha]


In [59]:
scf_oxidized_e, wfn = psi4.energy(level, return_wfn = True, molecule=mol_oxidized)
print('energy at the {} level of theory:\t{:.4f} [Ha]'.format(level, scf_oxidized_e))

energy at the B3LYP/6-31+G(d,p) level of theory:	-56.2742 [Ha]


In [60]:
# Attention !!!: The right way is to use Gibbs free energy
NHE = 4.28 # [V]
# F = 96485.3321233100184 # [C/mol] = [(J/V)/mol]
# 1 [J/mol] = 1.03636E-05 [eV]
F = 96485.3321233100184*1.03636E-05 # [e]
# 1 [Ha] = 13.6058*2 [eV], Ha = Hartree
Eox = (scf_oxidized_e - scf_neutral_e)*13.6058*2/(1*F) - NHE
print('Eox = {:.4f} [V]'.format(Eox))

Eox = 3.8150 [V]
